In [ ]:
import sys
sys.path.append('../src')
import numpy as np
from numpy import random
from scipy.stats import norm
from scipy.interpolate import CubicSpline
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras.backend as K
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import yaml

from fmda_models import XGB
import reproducibility

# Custom Loss Functions for Fuel Moisture Models

*Author:* Jonathon Hirschi

## Fuel Moisture Background

[Fuel moisture content](https://www.ncei.noaa.gov/access/monitoring/dyk/deadfuelmoisture) is a measure of the water content of burnable materials.

## Fuel Moisture Nonlinear Effect on Rate of Spread

[Rate of spread](https://www.nwcg.gov/course/ffm/fire-behavior/83-rate-of-spread#:~:text=The%20rate%20of%20spread%20is,origin%20quickly%20with%20great%20intensity.) (ROS) is a measure of the speed a fire moves (often units of m/s). The following image shows the nonlinear relationship between FM and ROS at a single spatial location, while holding other variables associated with ROS constant. Wildfire spreads most readily in dry fuels, as seen in the peak of the ROS curve at zero FM. The ROS drops off quickly as fuels get wetter, but then it levels off until the ROS is zero, or when the FM reaches the "extinction value". Below is an idealized rate of spread curve for fuel category 8, "Closed Timber Litter" ([NIFC Category Descriptions](https://gacc.nifc.gov/rmcc/predictive/Fire%20Behavior%20Fuel%20Model%20Descriptions.pdf)). This fuel is selected since it is closest to an idealized 10hr fuel. The fuel load contribution from dead 10hr fuels is the highest of any of the other fuel categories, and there is no contribution from live fuels.

<img src="../images/fuel8_ros_fm.png" alt="alt text" style="width: 500px;"/>

If the goal of training fuel moisture models is to get more accurate forecasts of wildfire ROS, it is intuitive that models should be trained directly on ROS. Instead of using loss functions on FM, why not construct a loss function directly with ROS? First, wildfire ROS is a complicated multifaceted conce3pt, and it is conceptually much cleaner to directly model fuel moisture and combine it with other observations to get reliable ROS estimates. Mathematically, there are issues related to the following two facts:

1. FM is highly correlated in time.
2. ROS reaches extinction value relatively quickly for dead fuels.

Since fuel moisture content is the "percent of the dry weight of that specific fuel" ([from NOAA](https://www.ncei.noaa.gov/access/monitoring/dyk/deadfuelmoisture#:~:text=Fuel%20moisture%20is%20a%20measure,content%20would%20be%20zero%20percent.)), this value can go over 100% for very wet fuels, since the water content can weight more than the underlying burnable material. The extinction value for tall grass, depicted above, reaches its extinction moisture value at roughly 25%. Thus, the ROS for tall grass with 25% FM would be the same as that of tall grass with 150% FM. In both cases, the ROS would be zero. 

This fact, combined with the temporal correlation of FM, makes it potentially undesirable to train FM models directly on ROS. Consider a case when the true FM content was 30% for tall grass. Model 1 predicts 25% FM and Model 2 predicts 150%. Both models would receive a loss of zero for that prediction, since both models predict zero ROS which matches the observed value. However, if atmospheric conditions led to the fuel drying out over time, the models would predict very different ROS within a few hours. Fuels with an FM of 25% would dry out relatively quickly compared to fuels with an FM of 150%, and thus the ROS would be nonzero in the former case much quicker than the latter. 

We first construct an idealized rate of spread curve from the source above. *Note:* an extinction moisture of about 25 is common for various fuel types.

In [ ]:
# Construct Idealized ROS curve from eyeballing plot
x = np.array([0, 5, 10, 15, 20, 25, 30, 35])
y = np.array([7.5, 4.3, 3.1, 2.6, 2.1, 1.4, 0, 0])*10**-3
xvals = np.linspace(start=0, stop=35, num=100)

ros_f = CubicSpline(x, y)
def ros(fm):
    r = ros_f(fm)
    r[fm>30]=0
    return r

plt.plot(xvals, ros(xvals), "red")
plt.xlabel("Fuel Moisture (%)")
plt.ylabel("Rate of Spread (m/s)")
plt.title("ROS Curve")
plt.grid()

## Exponential Weighting 

In [ ]:
fms = np.linspace(0, 35, 100)

fig, ax1 = plt.subplots()
# Plot the first line
ax1.plot(fms, ros(fms), 'r-', label='Rate of Spread')  
ax1.set_xlabel('Fuel Moisture (%)')
ax1.set_ylabel('Rate of Spread (m/s)', color='red')
ax1.tick_params('y', colors='r')
ax2 = ax1.twinx()

weights = np.ones(len(fms))
# ax2.plot(fms, weights, 'b-.', label='Equal Weight (unweighted)') 
ax2.plot(fms, weights, 'blue', label='Equal Weight (unweighted)') 

weights = tf.exp(tf.multiply(-0.01, fms))
ax2.plot(fms, weights, 'b--', label='$e^{-0.01}$ Weight') 

weights = tf.exp(tf.multiply(-0.025, fms))
ax2.plot(fms, weights, 'b-.', label='$e^{-0.025}$ Weight') 

weights = tf.exp(tf.multiply(-0.05, fms))
ax2.plot(fms, weights, 'b:', label='$e^{-0.05}$ Weight') 

weights = tf.exp(tf.multiply(-0.1, fms))
ax2.plot(fms, weights, 'b--', label='$e^{-0.1}$ Weight') 

ax2.set_ylabel('Weight', color='blue')
ax2.tick_params('y', colors='b')
ax2.set_ylim(0, 1.117)
fig.legend(loc="upper left", bbox_to_anchor=(1, .8))
ax1.grid(True)
plt.show()

In [ ]:
# Loss function with weights based on amplitude of y_true
def weighted_MSE(y_true,y_pred, val = -0.01):
    return K.mean(
        tf.multiply(
        tf.exp(tf.multiply(val, y_true)),
        tf.square(tf.subtract(y_pred, y_true))
    )
)

## Test Example

In [ ]:
df_all = pd.read_pickle("../data/rocky_2023_05-09.pkl")
df = df_all[df_all['stid'] == "CPTC2"]
df = df[
    (df['date'] >= '2023-06-01') &
    (df['date'] <= '2023-06-14')
]

plt.plot(df.date, df.fm)
plt.title("FM Observations at CPTC2 from 2023-06-01 through 2023-06-14")
plt.xticks(rotation=90)
plt.grid()

Now we train a simple XGBoost model on the first 13 days and predict the last one.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df[["Ed", "Ew"]], df['fm'], test_size=.2)

In [ ]:
X_train = df[["Ed", "Ew", "hour"]][df.date < '2023-06-13'] # get columns for model
y_train = df["fm"][df.date < '2023-06-13']
train_dates = df["date"][df.date < '2023-06-13']

X_test = df[["Ed", "Ew", "hour"]][df.date >= '2023-06-13'] # get columns for model
y_test = df["fm"][df.date >= '2023-06-13']
test_dates = df["date"][df.date >= '2023-06-13']

print(f"Training Observations: {y_train.shape[0]}")
print(f"Test Observations: {y_test.shape[0]}")

In [ ]:
with open('../models/params.yaml', 'r') as file:
    all_params = yaml.safe_load(file)

params = all_params["xgb"]
params

In [ ]:
reproducibility.set_seed(123)
model = XGB(loss='reg:squarederror',params=params)
model.fit(X_train, y_train)
fitted = model.predict(X_train)
preds = model.predict(X_test)

In [ ]:
plt.plot(df.date, df.fm, label = "FM Observed")
plt.plot(train_dates, fitted, label = "Fitted")
plt.plot(test_dates, preds, label = "Forecasts")
plt.title("FM Observations at CPTC2 from 2023-06-01 through 2023-06-14")
plt.xticks(rotation=90)
plt.legend()
plt.grid()

In [ ]:
# Summarise Error 
print(f"Test RMSE: {np.sqrt(mean_squared_error(y_test, preds))}")
print(f"Test Mean Bias: {np.mean(preds-y_test)}")

The RMSE shows middling model accuracy, but this metric treats negative and positive errors equally. If we examine the average bias of the model, the model is systematically overpredicting FMC in the prediction phase. Here, bias is defined simply as observed minus predicted.

In [ ]:
weights = tf.exp(tf.multiply(-0.1, y_train))

In [ ]:
reproducibility.set_seed(123)
model2 = XGB(params=params)
model2.fit(X_train, y_train, weights)
fitted2 = model2.predict(X_train)
preds2 = model2.predict(X_test)

In [ ]:
# Summarise Error 
print(f"Test RMSE: {np.sqrt(mean_squared_error(y_test, preds2))}")
print(f"Test Mean Bias: {np.mean(preds2-y_test)}")

In [ ]:
plt.plot(df.date, df.fm, label = "FM Observed")
plt.plot(train_dates, fitted2, label = "Fitted")
plt.plot(test_dates, preds2, label = "Forecasts")
plt.title("FM Observations at CPTC2 from 2023-06-01 through 2023-06-14")
plt.xticks(rotation=90)
plt.legend()
plt.grid()

In [ ]:
plt.plot(test_dates, y_test, label = "True")
plt.plot(test_dates, preds, label = "Unweighted")
plt.plot(test_dates, preds2, label = "Weighted")
plt.title("FM Observations at CPTC2 from 2023-06-01 through 2023-06-14")
plt.xticks(rotation=90)
plt.legend()
plt.grid()

## Predicting ROS

In [ ]:
plt.plot(ros(df.fm))

## References

* Open Wildland Fire Modeling E Community. https://wiki.openwfm.org/wiki/
* National Wildfire Coordinating Group (NWCG). https://www.nwcg.gov/course/ffm/
* *Dead Fuel Moisture*, NOAA National Centers for Environmental Information. https://www.ncei.noaa.gov/access/monitoring/dyk/deadfuelmoisture